In [16]:
from pymystem3 import Mystem

m = Mystem()

texts = []
scores = []
with open('texts_train.txt', 'r') as f:
    texts = list(map(lambda line: m.lemmatize(line.strip()), f.readlines()))

In [18]:
texts[0]

['сериал',
 ' ',
 'очень',
 ' ',
 'любить',
 ', ',
 'но',
 ' ',
 'академия',
 ' ',
 'и',
 ' ',
 'земля',
 ' ',
 'вызывать',
 ' ',
 'у',
 ' ',
 'я',
 ' ',
 'отторжение',
 ' ',
 'идея',
 '... ',
 'не',
 ' ',
 'любить',
 ', ',
 'когда',
 ' ',
 'принижать',
 ' ',
 'ценность',
 ' ',
 'человек',
 ' ',
 'как',
 ' ',
 'личность',
 ', ',
 'даже',
 ' ',
 'не',
 ' ',
 'смотря',
 ' ',
 'на',
 ' ',
 'ошибка',
 ' ',
 'личность',
 '...',
 '\n']

In [42]:
with open('scores_train.txt', 'r') as f:
    scores = list(map(int, f.readlines()))

In [185]:
with open('features', 'r') as f:
    features = list(map(lambda s: s.strip('\n'), f.readlines()))

In [186]:
features

['очень',
 'к',
 'перечитывать',
 'режиссер',
 'каждый',
 ' ',
 'раз',
 'один',
 'из',
 '(',
 'то',
 'мой',
 '.',
 '!!!',
 'книга',
 'отличный',
 'любимый',
 'не',
 'но',
 '!',
 'впрочем',
 'плохо',
 'рекомендовать',
 'пытаться',
 'однако',
 'вроде',
 'никакой',
 'зритель',
 'прекрасный',
 'как-то',
 'американский',
 'довольно',
 'плохой',
 'вполне',
 'замечательный',
 'неплохой',
 'супер',
 'великолепный',
 'потрясать',
 'один',
 'перечитывать',
 '(',
 'великолепный',
 '.',
 'замечательный',
 'плохо',
 'потрясать',
 'любимый',
 'дочитывать',
 'никакой',
 'но',
 'плохой',
 'книга',
 'отличный',
 'очень',
 'полный',
 '!',
 'не',
 ':)',
 '(:',
 ':(',
 ')))',
 '(((',
 ')',
 'великолепный',
 'отрываться',
 'ужас',
 'замечательный',
 'зачем',
 'плохо',
 'никакой',
 'потрясать',
 'любимый',
 'плохой',
 'полный',
 'скучно',
 'отличный',
 '?']

In [7]:
import numpy as np

baseline_score = np.mean(scores)

In [8]:
baseline_score

7.8584

In [83]:
from collections import Counter

all_used_words = Counter()
for l in texts:
    for w in l:
        if w == ' ':
            all_used_words[' '] += 1
            continue
        if w.strip() in ['», «', 'htm', 'exler', 'films', '://', 'http', 'ru', 'www', '»','«', '/', '"', ':', '«', '»']:
            continue
        if w.isdigit():
            continue
        if w.strip() == '':
            continue
        all_used_words[w.strip()] += 1

In [14]:
with open('input_3.txt', 'r') as fin, open('output.txt', 'w') as fout:
    for line in fin:
        print(8, file=fout)

In [84]:
len(all_used_words)

46431

In [24]:
len(texts)

20000

In [187]:
ft = [w for w, c in all_used_words.most_common(200)]
ft += features
features = list(set(ft))


In [188]:
N = len(features)
N

229

In [175]:
def transform_to_features(line):
    featured_text = np.zeros(N)
    text = m.lemmatize(line.strip())
    for i, el in enumerate(features):
        w = el
        featured_text[i] = sum([1 if w == word.strip() or w == word else 0 for word in text]) / len(text)
    return featured_text

In [189]:
featured_dataset = []
for text in texts:
    featured_dataset.append(transform_to_features(' '.join(text)))

In [88]:
from sklearn.svm import SVC

clf = SVC(gamma=2, C=1)

In [89]:
print(len(featured_dataset), len(scores))

clf.fit(featured_dataset, scores)

20000 20000


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [90]:
clf.score(featured_dataset[10000:11000], scores[10000:11000])

0.302

In [50]:
def transform_to_features(line):
    featured_text = np.zeros(200)
    text = m.lemmatize(line.strip())
    for i, el in enumerate(features):
        w, cnt = el
        featured_text[i] = sum([1 if w == word else 0 for word in text]) / len(text)
    return featured_text

In [93]:
with open('input_4.txt', 'r') as fin, open('output.txt', 'w') as fout:
    for line in fin:
        print(*clf.predict([transform_to_features(line)]), file=fout)

In [92]:
important = {f:0 for f, _ in features}
for vector in clf.support_vectors_:
    for i in range(200):
        if vector[i] != 0:
            important[features[i][0]] += abs(vector[i])
print(sorted(important.items(), key=lambda x: x[1]))
len(important)

[('–', 2.110278735541144), ('выходить', 3.032795962437159), ('—', 3.2405277653828017), ('картина', 3.3034033852395486), ('их', 3.40044284905941), ('режиссер', 3.882220380673607), ('оказываться', 4.24485893337861), ('персонаж', 4.362402972759865), ('роль', 4.543586459665923), ('день', 4.717790911279032), ('ее', 4.72929533874446), ('здесь', 4.7360012093377755), ('делать', 4.746050266249919), ('жить', 5.0731552857945115), ('друг', 5.123478173194418), ('сделать', 5.1993391320444875), ('дело', 5.2067077163238125), ('тут', 5.312784247432608), ('казаться', 5.537662306011617), ('находить', 5.652731786998256), ('давать', 5.716679132680492), ('),', 5.724378372971381), ('где', 5.777331811167201), ('именно', 5.851841177752793), ('ли', 5.985049688900518), ('потому', 5.986255259974915), ('ты', 6.084063948085121), ('при', 6.128875737940966), ('там', 6.246967858779574), ('два', 6.462835268300307), ('говорить', 6.530493210576587), ('под', 6.671577190050796), ('уж', 6.724466714914291), ('большой', 6.757

200

In [113]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=7)

In [114]:
clf.fit(featured_dataset, scores)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [115]:
clf.score(featured_dataset[10000:11000], scores[10000:11000])

0.289

In [116]:
with open('input_5.txt', 'r') as fin, open('output.txt', 'w') as fout:
    for line in fin:
        print(*clf.predict([transform_to_features(line)]), file=fout)

In [117]:
sorted(list(zip(clf.feature_importances_, features)), key=lambda x: x[0])[-20:]

[(0.008220824277000126, ('впрочем', 363)),
 (0.008536273555216592, ('плохо', 364)),
 (0.010166967010984254, ('рекомендовать', 610)),
 (0.010353364609337098, ('пытаться', 404)),
 (0.01242371763183187, ('однако', 470)),
 (0.013626006419369065, ('вроде', 515)),
 (0.014182743498433938, ('никакой', 428)),
 (0.015490109770095436, ('зритель', 494)),
 (0.017812891329699647, ('прекрасный', 510)),
 (0.02108231590510827, ('как-то', 434)),
 (0.02246586834483264, ('американский', 583)),
 (0.02336849013210757, ('довольно', 558)),
 (0.02437784701248681, ('плохой', 478)),
 (0.028418375264236774, ('»,', 590)),
 (0.032394487422238875, ('вполне', 504)),
 (0.04569775616868022, ('замечательный', 579)),
 (0.04914400304836285, ('неплохой', 369)),
 (0.051445362979390466, ('супер', 382)),
 (0.05925875046688445, ('великолепный', 473)),
 (0.06719526868689822, ('потрясать', 422))]

In [198]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=7, n_estimators=100)

In [204]:
regr.fit(featured_dataset[:-1000], scores[:-1000])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [200]:
regr.score(featured_dataset[10000:11000], scores[10000:11000])

0.17024929511620235

In [201]:
with open('input_9.txt', 'r') as fin, open('output.txt', 'w') as fout:
    for line in fin:
        b = regr.predict([transform_to_features(line)])[0]
        print(int(round(b)), file=fout)

In [194]:
sorted(list(zip(regr.feature_importances_, features)), key=lambda x: x[0])

[(0.0, '(:'),
 (1.1577332456210003e-05, 'несколько'),
 (1.2450522413443984e-05, 'надо'),
 (4.740735877581182e-05, 'про'),
 (4.813853108526823e-05, 'роль'),
 (5.550593875394062e-05, 'рекомендовать'),
 (5.716309100467467e-05, 'выходить'),
 (8.060813672670882e-05, 'взгляд'),
 (8.73513001981714e-05, 'довольно'),
 (9.272133858876324e-05, 'однако'),
 (0.0001003730543128855, 'или'),
 (0.00010266683092659276, 'во'),
 (0.00011861400641366552, 'жить'),
 (0.000121297513563617, 'правда'),
 (0.00012669861093534272, 'режиссер'),
 (0.00016719381726809835, 'впечатление'),
 (0.00016929145324551874, 'давать'),
 (0.0001751818668706777, ')))'),
 (0.00018055216195914683, 'день'),
 (0.00018856012972152033, 'часть'),
 (0.00019138227152140476, 'тоже'),
 (0.00019439232690584668, ':)'),
 (0.00019531319265005505, 'прекрасный'),
 (0.00020430814727663056, 'находить'),
 (0.00020587141295703263, 'новый'),
 (0.0002075344441783848, 'где'),
 (0.0002127171716933672, 'здесь'),
 (0.0002176410788761605, '—'),
 (0.000231264

In [205]:
from sklearn.metrics import r2_score

r2_score(scores[-1000:], regr.predict(featured_dataset[-1000:]))

0.10669635023326185